#데이터 로드 

In [7]:
!mkdir Data 
%cd ./Data 
!gdown https://drive.google.com/u/0/uc?id=1SZ5j8SJTCqQQuNHTMDHXSOMShKUvB479&export=download
!unzip open.zip 
!rm -f open.zip
%cd /content

mkdir: cannot create directory ‘Data’: File exists
/content/Data
Downloading...
From: https://drive.google.com/u/0/uc?id=1SZ5j8SJTCqQQuNHTMDHXSOMShKUvB479
To: /content/Data/open.zip
100% 2.56G/2.56G [00:28<00:00, 89.5MB/s]


In [28]:
import os 
import warnings 

import cv2 
import pandas as pd 
from glob import glob 
from tqdm import tqdm 
import random 
import matplotlib.pyplot as plt 
import numpy as np 
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers 

# 1단계 이미지 디렉토리 로드 

In [29]:
img_dirs = sorted(glob('./Data/train/*.jpg'))[:2000]
train_csv = pd.read_csv('./Data/train.csv')
img_size = 224 


# 2단계 이미지 데이터 로드 및 전처리 

In [ ]:

def img_preprocess(img):
    global img_size 
    img = cv2.resize(img,dsize=(img_size,img_size))
    img = img/255.
    img = img.astype(np.float16)
    return img 

def img_load(img_dir):
    img = cv2.imread(img_dir)
    img = img_preprocess(img)
    return img 

imgs = [img_load(img_dir) for img_dir in tqdm(img_dirs)]
imgs = np.array(imgs)

# 3단계 라벨 데이터 로드 

In [30]:
#라벨 데이터 정제 
label_unique = np.unique(train_csv['artist'])
label_decoder = {key:value for key,value in enumerate(label_unique)}
label_encoder = {value:key for key,value in enumerate(label_unique)}

labels = train_csv['artist'].apply(lambda x : label_encoder[x]).values[:2000]

# Train-Valid 분리 

In [42]:
valid_ratio = 0.8 
valid_index = int(len(imgs)*valid_ratio)
train_x = imgs[:valid_index]
valid_x = imgs[valid_index:]

train_y = labels[:valid_index]
valid_y = labels[valid_index:]